# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

SyntaxError: invalid syntax (<ipython-input-4-71a7702f86c3>, line 3)

In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
document = ET.parse( './data/mondial_database.xml' )

In [6]:
import pandas as pd

In [8]:
root = document.getroot()

In [9]:
df = pd.DataFrame(columns=('country', 'inf_mortality'))
df_temp = pd.DataFrame(columns=('country', 'inf_mortality'))
for country in root.findall('country'):
    name = country.find('name').text
    inf_mortality = country.find('infant_mortality')
    if inf_mortality is not None:
        infant_mortality=inf_mortality.text
        df_temp=pd.DataFrame({'country':[name],
                    'inf_mortality':[infant_mortality]})
        #df_temp
        df = pd.concat([df,df_temp])
df["inf_mortality"] =pd.to_numeric(df["inf_mortality"])
df.sort_values(['inf_mortality'], ascending=[False]).head(10)

,country,inf_mortality
0,Western Sahara,145.82
0,Afghanistan,117.23
0,Mali,104.34
0,Somalia,100.14
0,Central African Republic,92.86
0,Guinea-Bissau,90.92
0,Chad,90.30
0,Niger,86.27
0,Angola,79.99
0,Burkina Faso,76.80


In [10]:
df = pd.DataFrame(columns=('city', 'population'))
df_temp = pd.DataFrame(columns=('city', 'population'))
for city in root.iter('city'):
    name = city.find('name').text
    pop = city.find('population')
    if pop is not None:
        population = pop.text
        df_temp=pd.DataFrame({'city':[name],
                    'population':[population]})
        df = pd.concat([df,df_temp])

df["population"] =pd.to_numeric(df["population"])
df.sort_values(['population'], ascending=[False]).head(10)

,city,population
0,Seoul,10229262
0,Mumbai,9925891
0,São Paulo,9412894
0,Jakarta,8259266
0,Shanghai,8205598
0,Ciudad de México,8092449
0,Moskva,8010954
0,Tokyo,7843000
0,Beijing,7362426
0,Delhi,7206704


In [11]:
df = pd.DataFrame(columns=('country', 'ethnicity','population'))
df_temp = pd.DataFrame(columns=('country', 'ethnicity','population'))
for country in root.findall('country'):
    population = country.find('population[last()]')
    country_name = country.find('name').text
    if population is not None:
        pop = (float(population.text))
        #print(pop)
    for ethnicgroups in country.iter('ethnicgroup'):
        name = ethnicgroups.text
        percent = ethnicgroups.get('percentage')
        ethnic_pop = float(percent)*pop
        df_temp=pd.DataFrame({'country':[country_name],
                                  'ethnicity':[name],
                                  'population':[ethnic_pop]})
        df = pd.concat([df,df_temp])
df.sort_values(['population'], ascending=[False]).head(10)

,country,ethnicity,population
0,China,Han Chinese,1.245059e+11
0,India,Indo-Aryan,8.718156e+10
0,India,Dravidian,3.027137e+10
0,United States,European,2.549581e+10
0,Nigeria,African,1.626516e+10
0,Bangladesh,Bengali,1.467769e+10
0,Japan,Japanese,1.265342e+10
0,Russia,Russian,1.146462e+10
0,Indonesia,Javanese,1.134560e+10
0,Brazil,European,1.088867e+10


In [12]:
df = pd.DataFrame(columns=('river', 'country','length'))
df_temp = pd.DataFrame(columns=('river', 'country','length'))
for river in root.iter('river'):
    name = river.get('id')
    country = river.get('country')
    if name is not None:
        length = river.find('./length')
        if length is not None:
            #print (length)
            len1 = length.text
            df_temp=pd.DataFrame({'river':[name],
                                  'country':[country],
                                  'length':[len1]})
            df = pd.concat([df,df_temp])

df["length"] =pd.to_numeric(df["length"])
df.sort_values(['length'], ascending=[False]).head(1)

,country,length,river
0,CO BR PE,6448.0,river-Amazonas


In [13]:
df = pd.DataFrame(columns=('lake', 'country','area'))
df_temp = pd.DataFrame(columns=('lake', 'country','area'))
for lake in root.iter('lake'):
    name = lake.get('id')
    country = lake.get('country')
    if name is not None:
        area = lake.find('./area')
        if area is not None:
            #print (length)
            area1 = area.text
            df_temp=pd.DataFrame({'lake':[name],
                                  'country':[country],
                                  'area':[area1]})
            df = pd.concat([df,df_temp])

df["area"] =pd.to_numeric(df["area"])
df.sort_values(['area'], ascending=[False]).head(1)

,area,country,lake
0,386400.0,R AZ KAZ IR TM,lake-KaspischesMeer


In [14]:
df = pd.DataFrame(columns=('airport', 'country','elevation'))
df_temp = pd.DataFrame(columns=('airport', 'country','elevation'))
for airport in root.iter('airport'):
    name = airport.get('iatacode')
    country = airport.get('country')
    if name is not None:
        elevation = airport.find('./elevation')
        if elevation is not None:
            #print (length)
            elevation1 = elevation.text
            df_temp=pd.DataFrame({'airport':[name],
                                  'country':[country],
                                  'elevation':[elevation1]})
            df = pd.concat([df,df_temp])

df["elevation"] =pd.to_numeric(df["elevation"])
df.sort_values(['elevation'], ascending=[False]).head(1)

,airport,country,elevation
0,LPB,BOL,4063.0
